In [1]:
import os
mingw_path="C:\\Program Files\\mingw-w64\\x86_64-7.2.0-posix-seh-rt_v5-rev1\mingw64\\bin"
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

In [2]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.metrics import  accuracy_score
from sklearn.preprocessing import OneHotEncoder
from numpy import column_stack
import pickle


C:\Users\anurag dubey\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [98]:
df1 =pd.read_csv("D:/Hackethon/LoanPrediction/train.csv",na_values=["NaN","nan"," "])

df2  = pd.read_csv("D:/Hackethon/LoanPrediction/test.csv",na_values=["NaN","nan"," "])
df2["Loan_Status"] ="N"
df3 = pd.concat((df1,df2))
df3 = df3.drop("Loan_ID",1)
df3["Dependents"]  =df3["Dependents"].apply(lambda X:"3" if X=="3+" else X)
df3["Loan_Amount_Term"]  =df3["Loan_Amount_Term"].astype(str)
#df3["Loan_Amount_Term"] = df3["Loan_Amount_Term"].apply(lambda X:X if X=="360.0" or X=="180.0" else "Other")

In [99]:
ApplicantIncome = df3["ApplicantIncome"]
CoapplicantIncome = df3["CoapplicantIncome"]
LoanAmount = df3["LoanAmount"]
df3.drop("ApplicantIncome",axis =1,inplace =True)
df3.drop("CoapplicantIncome",axis =1,inplace =True)
df3.drop("LoanAmount",axis =1,inplace =True)

In [100]:
df3ConData = pd.DataFrame({"ApplicantIncome":ApplicantIncome,"CoapplicantIncome":CoapplicantIncome,"LoanAmount":LoanAmount})

In [101]:
df3ConData["LoanAmount"] = df3ConData["LoanAmount"]*1000

In [107]:
df3ConData["LoanAmount"] = df3ConData["LoanAmount"].fillna(0)

df3ConData["LoanAmount"]= df3ConData["LoanAmount"].astype(int)
# df3ConData["LoanAmount"] = df3ConData["LoanAmount"]+1
df3ConData["LogLoanAmount"]  =df3ConData["LoanAmount"].apply(np.log)
print (df3ConData["LogLoanAmount"].head())

0     0.000000
1    11.759793
2    11.097425
3    11.695255
4    11.856522
Name: LogLoanAmount, dtype: float64


In [108]:
df3ConData =df3ConData.values
print

In [109]:
df3 = df3.values

In [110]:
X = df3[:,0:8]
Y =  df3[:,8]

In [111]:
X = X.astype(str)
Columns =[]
labelEncoder =LabelEncoder()
for i in range(0,7):
    feature = labelEncoder.fit_transform(X[:,i])
    feature = feature.reshape(X.shape[0],1)
    oneHotEncoder = OneHotEncoder(sparse=False)
    Feature_encoded = oneHotEncoder.fit_transform(feature)
    Columns.append(Feature_encoded)
X_Encoded = column_stack(Columns)
    

In [112]:
XTrain_new= np.concatenate((X_Encoded,df3ConData),axis=1)

In [113]:
print(XTrain_new.shape)

(981, 36)


In [114]:
TrainS = XTrain_new[:614,:]
TestS  =XTrain_new[614:,:]

In [115]:
print(TrainS.shape)
print(TestS.shape)

(614, 36)
(367, 36)


In [55]:
Y_New = Y[:614]

In [56]:
Y_Trans = labelEncoder.fit_transform(Y_New)

In [63]:
XTrain,XTest,YTrain,YTest = train_test_split(TrainS,Y_Trans,test_size = 0.2,random_state = 7)

In [64]:
model = XGBClassifier()
model.fit(XTrain,YTrain)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [66]:
pred= model.predict(XTest)
print (accuracy_score(YTest,pred))

0.829268292683


In [67]:
pred= model.predict(TestS)

In [68]:
print (pred)

[1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 0 1 1
 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1
 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1]


In [69]:
df4  = pd.read_csv("D:/Hackethon/LoanPrediction/ss.csv",na_values=["NaN","nan"," "])
df2  = pd.read_csv("D:/Hackethon/LoanPrediction/test.csv",na_values=["NaN","nan"," "])

In [70]:
df4["Loan_ID"] = df2["Loan_ID"]
df4["Loan_Status"] = pred

In [71]:
print(df4.shape[0])

367


In [72]:
df4.to_csv("D:/Hackethon/LoanPrediction/sub2.csv", index=False)

In [73]:
pickle.dump(model, open("D:\\Hackethon\\XGmodel2.dat","wb"))